***This program for change colorspace From HDMI Input and send to HDMI ouput***
>Connection
- connect HDMI Input with Output HDMI on user PC
- connect HDMI Output to LCD monitor
- Open Jupyter notebook on your PYNQ-ZU
- Run The Program

import base overlay and load the bitstream file

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("base.bit")

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

Start and cconfigure HDMI Input and Output

In [ ]:
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

bound HDMI Output to HDMI Input

In [ ]:
hdmi_in.tie(hdmi_out)

Read image from HDMI Input and convert color from BGR to laplacian grayscale and sent to HDMI out as BGR

In [ ]:
import time
import cv2
import numpy as np

numframes = 600
start = time.time()

grayscale = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                       dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                    dtype=np.uint8)

start = time.time()

for _ in range(numframes):
    inframe = hdmi_in.readframe()
    cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale)
    cv2.Laplacian(grayscale, cv2.CV_8U, dst=result)

    outframe = hdmi_out.newframe()
    cv2.cvtColor(result, cv2.COLOR_GRAY2BGR,dst=outframe)
    hdmi_out.writeframe(outframe)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Close HDMI connection

In [ ]:
hdmi_out.close()
hdmi_in.close()